# WDSS Football Forecasting Competition





<img src="logo_subtitle.png" width=500 height=500 />

## Our Sponsor

<img src="mustard-systems-logo.png" width=500 height=500 />

## The Competition

- Predicting the scores of football matches isn't a new idea 


- People try their luck at bookies daily with nothing more than guesses, however at WDSS a more data-driven approach is preferred 

- The competition aims to draw together the best implementations of these with hopes of finding the best model possible 

## The Format

- Weekly predictions on the **scores** of each Premier League match


- Each week the predictions must be **ranked** from 1 to 10 in terms of confidence

- Each contestor's predictions will be scored based on their weighted **mean squared error** and converted into points using a monotonously-decreasing rule 

## The Rules

- All weekly submissions must be accompanied by a model created in Python Jupyter Notebook (.ipynb file) or R Markdown notebooks (.Rmd) 

- Scores predicted must match the output of your model (we will check this and disqualify inconsistent submissions) 



- There will be some flexibility in what constitutes as a model, but please refer to our demo model as a benchmark 



- Have Fun! 

## The Winner & Prize

- The competition will be split across Term 1 & 2 with a prize pool of £700 for each iteration 


- This prize pool will be split between the top competitors as ranked by the accuracy of their predictions 


- Additionally, a £100 prize will be awarded for the model that displays the most ingenuity, creativity, and good statistical practice 

# Getting Started: lets build a demo model

- Today we will guide you in building a baseline model for our upcoming Premier League forecasting competition 

- This model will NOT win the competition for you, but it will help point you in the right direction

## Start out with our imports

In [ ]:
# Dependencies
from scipy.stats import poisson, skellam
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
from matplotlib.ticker import AutoMinorLocator
import seaborn as sns
import pandas as pd
import statsmodels.api as sm
import statsmodels.formula.api as smf

## Gathering our data

In [ ]:
#### FUNCTION TO RETRIEVE PREMIER LEAGUE DATA ####
def get_premier_league_data(start_year):
    season = str(start_year)[-2:] + str(start_year + 1)[-2:]
    data = pd.read_csv("http://www.football-data.co.uk/mmz4281/" + season + "/E0.csv") 
    return data

In [ ]:
# Get data from the 2018/2019 season
data = get_premier_league_data(2018)
data.head()

## Light cleaning

In [ ]:
# Filtering columns of interest
columns = ["HomeTeam", "AwayTeam", "FTHG", "FTAG", "FTR"]
data = data[columns]

# Renaming columns
data = data.rename(
    columns={"FTHG": "HomeGoals", "FTAG": "AwayGoals", "FTR": "Result"}
)

In [ ]:
# Remove final week of fixtures 

In [ ]:
data.head()

## Simple analysis: Home team advantage?


In [ ]:
# Compute the average number of home and away goals 

## Towards a match prediction model 

- One way to predict the match score is to consider the number of goals scored by each team


- We will denote the number of goals scored by a particular team in a match by $y_i$, where $i$ indicates the particular match


- Furthermore, we will use *regression analysis* to model $\mathbb{E}[y_i | X_i]$ 

## The Poisson distribution

- The Poisson distribution is often used to model the probability distribution of *count events* (that is, the same event happening a specific number of times in a fixed time frame)


- It is a *discrete* distribution parametarized by a mean constant rate of occurences $\lambda$

- It assumes that event occurances within the interval are *independent* of one another

- It can be especially useful to model the number of goals we expect a team to score

## The Poisson distribution: an example

In [ ]:
# Here we simulate 1 million samples from a Poisson distribution with a mean of two
x = np.random.poisson(2, 1000000)
plt.hist(x, 14, density=True)
plt.show()

## Formatting our data

In [ ]:
#### PREPARE THE DATASET ####

# Separate home goals data
home_goals = data[["HomeTeam", "AwayTeam", "HomeGoals"]]
home_goals = home_goals.assign(home=1)
home_goals = home_goals.rename(
    columns={"HomeTeam": "team",
             "AwayTeam": "opponent", 
             "HomeGoals": "goals"}
)

# Separate away goals data 

In [ ]:
# Concatenating into training data 
training_data = pd.concat([home_goals, away_goals])
training_data.head()

## The Poisson regression model

- To predict the number of goals each team scores, we will fit the following model using match-level data


$$y_i \sim Poisson(\lambda_i)$$


$$\ln (\lambda_i) = X_i\beta$$

- Here, $\lambda_i$ is the mean number goals scored by the home team in match $i$, which we aim to predict using variables $X_i$

In [ ]:
# Building the model
# Poisson Regression: log-linear model
poisson_model = smf.glm(
    formula="goals ~ ???",
    data=training_data,
    family=sm.families.Poisson() 
).fit()

In [ ]:
# Get a statistical summary of the poisson model
poisson_model.summary()

## A note on dummies, coefficients, and interpretation

## Computing the expected goals for each team

In [ ]:
# Create feature data for home and away team for the match
def create_X(home_team, away_team):
    X_home = pd.DataFrame(data={"team": home_team,
                                "opponent": away_team,
                                "home": 1
                                }, index=[1])
    
    # Creating DataFrame for away team features 
    return X_home, X_away

In [ ]:
def predict_avg_goals(X_home, X_away, model):
    # Predict the mean number of goals for home team
    home_goals_avg = model.predict(X_home) 
    
    # Predict the mean number of goals for away team
    away_goals_avg = model.predict(X_away) 
    
    return home_goals_avg, away_goals_avg

In [ ]:
# Predicting expected goals for Chelsea Man City match
X_home, X_away =  create_X('Chelsea', 'Man City')
expected_goals = predict_avg_goals(X_home, X_away, poisson_model)
print('Expected Home Goals: ', expected_goals[0].values[0])
print('Expected Away Goals: ', )

## Computing the distribution of match scores

In [ ]:
#### MATCH SCORE FORECASTING FUNCTION ####
def predict_score_pmf(X_home, X_away, model, max_goals):
    # Predict the average number of goals for home  and away teams
    expected_goals = predict_avg_goals(X_home, X_away, model)
    home_goals_avg = expected_goals[0]
    away_goals_avg = expected_goals[1]

    # Compute marginal distribution for home goals
    home_goals_pmf = [poisson.pmf(i, home_goals_avg)
                      for i in range(0, max_goals + 1)]

    # Compute marginal distribution for away goals 

    # Compute joint distribution for match score as outer product
    joint_pmf = np.outer(np.array(home_goals_pmf),
                         np.array(away_goals_pmf))
    
    return joint_pmf

In [ ]:
# Predicting distribution of match scores
X_home, X_away =  create_X('Chelsea', 'Man City')
score_pmf = predict_score_pmf(X_home, X_away, poisson_model, 8)
score_pmf = score_pmf.round(3)

# Visualizing distribution in a heatmap
f, axs = plt.subplots(figsize=(8, 6))
sns.heatmap(score_pmf, annot=True)
plt.title("Chelsea v.s Man City score forecast")
plt.xlabel("Man City goals")
plt.ylabel("Chelsea goals")
plt.show()

## Forecasting the distribution mode

In [ ]:
def predict_score(X_home, X_away, model):
    # Predicting distribution of match scores
    score_pmf = predict_score_pmf(X_home, X_away, model,16)
    score_pmf = score_pmf.round(3)
    
    # Computing distribution mode
    home_goals_mode = np.argmax(score_pmf) // (17)
    away_goals_mode = np.argmax(score_pmf) % (17)
    score_pred = (home_goals_mode, away_goals_mode)
    return score_pred

In [ ]:
# Predicting Chelsea Man City match score
X_home, X_away =  create_X('Chelsea', 'Man City')
score_pred =  predict_score(X_home, X_away, poisson_model)
print('Score Forecast: ', score_pred)

## Validating our predictions ... how well did we do?

- To find out how our model performed, it is important to devise some form of scoring metric


- In this workshop, we will use the mean squared error to asses this:
$$\text{MSE}(y, \hat y) \,=\, \frac{1}{N} \sum_{i=1}^{N} (y_i - \hat y_i)^2$$

In [ ]:
def mse(y_true, ypred):
    np.square(np.subtract(Y_true,Y_pred)).mean()

In [ ]:
def model_eval(X, model):
    # Predicting distribution of match scores
    #### LEFT AS AN EXERCISE ####
    pass

## Links to data sources

https://www.rotowire.com//soccer/tables/standings.php?league=EPL&length=total&season=2019

https://www.rotowire.com/soccer/league-table.php?season=2018

## Next Steps

- Build on the benchmark


- Cross-validation


- Dixon-Coles model


- Additional regressors (betting odds?)


- Alternative data sources (Twitter and NLP?)


- Participate in the competition.